In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

2023-08-26 17:04:08.877757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset=pd.read_csv('/Users/uditaggarwal/Downloads/GOOG.csv')
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-28,13.561418,13.623934,13.215465,13.307371,13.307371,129648138
1,2010-01-29,13.411979,13.474245,13.091181,13.199027,13.199027,166240784
2,2010-02-01,13.315092,13.345229,13.207993,13.275739,13.275739,90369462
3,2010-02-02,13.324058,13.324058,13.140994,13.228417,13.228417,164462143
4,2010-02-03,13.167395,13.501892,13.156436,13.470011,13.470011,120413655


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1258 non-null   object 
 1   Open       1258 non-null   float64
 2   High       1258 non-null   float64
 3   Low        1258 non-null   float64
 4   Close      1258 non-null   float64
 5   Adj Close  1258 non-null   float64
 6   Volume     1258 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 68.9+ KB


In [4]:
dataset=dataset.dropna()
trainset=dataset.iloc[:,4:5].values

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1258 non-null   object 
 1   Open       1258 non-null   float64
 2   High       1258 non-null   float64
 3   Low        1258 non-null   float64
 4   Close      1258 non-null   float64
 5   Adj Close  1258 non-null   float64
 6   Volume     1258 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 68.9+ KB


In [6]:
scale=MinMaxScaler(feature_range=(0,1))
trainset=scale.fit_transform(trainset)
trainset.shape

(1258, 1)

In [7]:
W_train=[]
Y_train=[]
for i in range(60,1234):
    W_train.append(trainset[i-60:i,0])
    Y_train.append(trainset[i,0])
    
W_train,Y_train=np.array(W_train),np.array(Y_train)

In [8]:
W_train=np.reshape(W_train,(W_train.shape[0],W_train.shape[1],1))
W_train.shape

(1174, 60, 1)

In [9]:
technique=Sequential()

technique.add(LSTM(units=100,return_sequences=True,input_shape=(W_train.shape[1],1)))
technique.add(Dropout(0.2))

technique.add(LSTM(units=100,return_sequences=True))
technique.add(Dropout(0.2))

technique.add(LSTM(units=100,return_sequences=True))
technique.add(Dropout(0.2))

technique.add(LSTM(units=100,return_sequences=False))
technique.add(Dropout(0.2))

technique.add(Dense(units=1))
technique.compile(optimizer='adam',loss="mean_squared_error")

2023-08-26 17:04:33.352670: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
datatraining=technique.fit(W_train,Y_train,epochs=20,batch_size=32,verbose=2)

Epoch 1/20
37/37 - 7s - loss: 0.0192 - 7s/epoch - 192ms/step
Epoch 2/20
37/37 - 4s - loss: 0.0046 - 4s/epoch - 101ms/step
Epoch 3/20
37/37 - 4s - loss: 0.0042 - 4s/epoch - 100ms/step
Epoch 4/20
37/37 - 4s - loss: 0.0037 - 4s/epoch - 103ms/step
Epoch 5/20
37/37 - 4s - loss: 0.0035 - 4s/epoch - 100ms/step
Epoch 6/20
37/37 - 4s - loss: 0.0033 - 4s/epoch - 101ms/step
Epoch 7/20
37/37 - 4s - loss: 0.0029 - 4s/epoch - 100ms/step
Epoch 8/20
37/37 - 4s - loss: 0.0028 - 4s/epoch - 99ms/step
Epoch 9/20
37/37 - 4s - loss: 0.0032 - 4s/epoch - 101ms/step
Epoch 10/20
37/37 - 4s - loss: 0.0026 - 4s/epoch - 100ms/step
Epoch 11/20
37/37 - 4s - loss: 0.0027 - 4s/epoch - 99ms/step
Epoch 12/20
37/37 - 4s - loss: 0.0027 - 4s/epoch - 100ms/step
Epoch 13/20
37/37 - 4s - loss: 0.0029 - 4s/epoch - 101ms/step
Epoch 14/20
37/37 - 4s - loss: 0.0024 - 4s/epoch - 101ms/step
Epoch 15/20
37/37 - 4s - loss: 0.0023 - 4s/epoch - 101ms/step
Epoch 16/20
37/37 - 4s - loss: 0.0024 - 4s/epoch - 101ms/step
Epoch 17/20
37/37 -

In [ ]:
plt.plot(datatraining.history['loss'])
plt.title('Training model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()

In [ ]:
testset=pd.read_csv('/Users/uditaggarwal/Downloads/GOOG.csv')
testset=testset.dropna()
testset=testset.iloc[:,4:5]
Y_test=testset.iloc[60:,0:].values
inputclose=testset.iloc[:,0:].values
inputclose_scaled=scale.transform(inputclose)
inputclose_scaled.shape
W_test=[]
length=len(testset)
time_step=60
for i in range(time_step,length):
    W_test.append(inputclose_scaled[i-time_step:i,0])
W_test=np.array(W_test)
W_test=np.reshape(W_test,(W_test.shape[0],W_test.shape[1],1))
W_test.shape

In [ ]:
Y_predict=technique.predict(W_test)
Y_predict

In [ ]:
predicted_price=scale.inverse_transform(Y_predict)

In [ ]:
plt.plot(Y_test,color='green',label='Actual stock prices')
plt.plot(predicted_price,color='red',label='Predicted stock price')
plt.title('Samsung stock price prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()